# Weather data

importing libraries

In [104]:
import requests
import json
import pandas as pd
import time
import numpy as np
import pymysql
from datetime import date
from datetime import timedelta

In [90]:
today = (date.today()).strftime("%Y-%m-%d")
tomorrow = (date.today() + timedelta(days=1)).strftime("%Y-%m-%d")

In [89]:
appID = owm_key

List of cities we are interested in

In [105]:
munic_country_list = ['Berlin,DE',
 'Borkum,DE',
 'Bremen,DE',
 'Cologne,DE',
 'Dortmund,DE',
 'Dresden,DE',
 'Düsseldorf,DE',
 'Emden,DE',
 'Erfurt,DE',
 'Frankfurt,DE',
 'Friedrichshafen,DE',
 'Hamburg,DE',
 'Hanover,DE',
 'Heide,DE',
 'Heligoland,DE',
 'Heringsdorf,DE',
 'Karlsruhe,DE',
 'Kassel,DE',
 'Koblenz,DE',
 'Leipzig,DE',
 'Halle (Saale),DE',
 'Mannheim,DE',
 'Memmingen,DE',
 'Munich,DE',
 'Osnabrück,DE',
 'Nuremberg,DE',
 'Paderborn,DE',
 'Rostock,DE',
 'Saarbrücken,DE',
 'Stuttgart,DE',
 'Weeze,DE']

We can't use json_normalize, because if there is no snow or rain we don't get a column for it.

Need to write a function for it

Looking for date and time, temperature, wind speed,probility of precipitation(pop), rain volume (rain), snow volume(snow)

https://openweathermap.org/forecast5

I'm just interested fo the forecast for the next day, because to get the most accurate data and I will call the weather API once a day. 

In [98]:
def getting_weather_from_json(data_json,munic_country):
    
    l_aux_getting_weather = []
    
    for data_point in data_json: 
        dic_aux_getting_weather = {}
        if tomorrow in data_point['dt_txt']:
            dic_aux_getting_weather['date_time'] = data_point['dt_txt']
            dic_aux_getting_weather['temperature'] = data_point['main']['temp']
            dic_aux_getting_weather['wind'] = data_point['wind']['speed']
            #have to use try except for the next 3 stats because they could be empty and not create an entry
            try: dic_aux_getting_weather['prob_perc'] = float(data_point['pop'])
            except: dic_aux_getting_weather['prob_perc'] = 0
            try: dic_aux_getting_weather['rain_vol'] = float(data_point['rain']['3h'])
            except: dic_aux_getting_weather['rain_vol'] = 0
            try: dic_aux_getting_weather['snow_vol'] = float(data_point['snow']['3h'])
            except: dic_aux_getting_weather['snow_vol'] = 0
            
            dic_aux_getting_weather['munic_country'] = munic_country
            l_aux_getting_weather.append(dic_aux_getting_weather)

    df_weather_data = pd.DataFrame(l_aux_getting_weather)
    return df_weather_data


Function to iterate over the list of cities

In [106]:
def calling_owm(munic_country_list,appID):
    df_weather1 = pd.DataFrame()
    for munic_country in munic_country_list:

        response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={munic_country}&appid={appID}&units=metric&lang=en')

        
        if response.status_code == 200:
            
            data_json = response.json()['list']
            
            df_weather2 = getting_weather_from_json(data_json,munic_country)
            df_weather1 = pd.concat([df_weather1, df_weather2], ignore_index=True)
                        
        time.sleep(10)
        
    return df_weather1


Calling the function

In [108]:
df_weather_data_full = calling_owm(munic_country_list,appID)

Example dataframe just for Berlin and Bremen the next day

In [109]:
df_weather_data_full

,date_time,temperature,wind,prob_perc,rain_vol,snow_vol,munic_country
0,2022-04-11 00:00:00,3.49,4.00,0.00,0,0,"Berlin,DE"
1,2022-04-11 03:00:00,2.14,3.61,0.00,0,0,"Berlin,DE"
2,2022-04-11 06:00:00,3.15,3.82,0.00,0,0,"Berlin,DE"
3,2022-04-11 09:00:00,7.90,3.94,0.03,0,0,"Berlin,DE"
4,2022-04-11 12:00:00,10.57,3.62,0.00,0,0,"Berlin,DE"
5,2022-04-11 15:00:00,11.48,3.43,0.00,0,0,"Berlin,DE"
6,2022-04-11 18:00:00,8.00,2.60,0.00,0,0,"Berlin,DE"
7,2022-04-11 21:00:00,5.36,2.52,0.00,0,0,"Berlin,DE"
8,2022-04-11 00:00:00,0.61,3.08,0.00,0,0,"Bremen,DE"
9,2022-04-11 03:00:00,-0.18,2.66,0.00,0,0,"Bremen,DE"


Changing the type of the date_time column

In [111]:
df_weather_data_full = df_weather_data_full.assign(date_time = lambda x: pd.to_datetime(x['date_time']))

# Upload data to cloud

setting up connection

In [113]:
user=user_id
password=password
host=endpoint
database=db_name
port=port


In [114]:
con= f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}'

In [25]:
df_weather_data_full.to_sql('weather', if_exists='append', con=con, index=False)